<a href="https://colab.research.google.com/github/colgate116/T_rex_game/blob/master/video_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!pip install ultralytics

Sun May 19 13:25:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26297 sha256=6777afccbf7719279ccd3ff2383216ca3532ababcbeb2b737fbd1fc37f1542f2
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [4]:
from ultralytics import YOLO
import cv2
import cvzone
from enum import Enum

In [5]:
class Color(Enum):
    # brg
    RED = (0, 0, 180)
    PURPLE = (180, 0, 180)
    ORANGE = (0, 150, 180)
    YELLOW = (0, 180, 180)
    GREEN = (0, 180, 0)
    BLUE = (180, 0, 0)

In [19]:
# Отрисовка рамки
def bounding_box(img, coordinates, class_id):
    """
    :param img:
    :param coordinates: (x1, y1, x2, y2)
    :param class_id: id of detected class
    :return:
    """
    # С масштабированием под разные разрешения
    if img:
        x1, y1, x2, y2 = coordinates
        w0 = 640
        w, h = x2 - x1, y2 - y1
        h1, w1, c = img.shape
        k = w1 / w0
        class_name = classes[class_id]
        colors = [color.value for color in Color]
        class_color = colors[class_id]
        cvzone.cornerRect(img, (x1, y1, w, h), l=int(min(w, h) * 0.25), t=int(k), rt=0,
                          colorC=class_color)
        cvzone.putTextRect(img, f"{class_name}", (x1, y1 - h1 // 144), scale=k, thickness=int(k), offset=0,
                          colorR=class_color)

# Загружаем модель

In [7]:
model_path = r"YOLOv8m_model.pt"
model = YOLO(f"{model_path}")
classes = model.names

#Видео для детекции

In [20]:
media_path = r"sharks.mp4"
media = cv2.VideoCapture(f"{media_path}")
frame_width = int(media.get(3))
frame_height = int(media.get(4))

# img = cv2.imread(f"{img_path}")
# success, img = video.read()

# Видео для сохранения

In [21]:
output_media = cv2.VideoWriter("Shark_detection_v2.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

In [22]:
while media.isOpened():
    success, img = media.read()

    if not success:
        exit(1)

    res = model(img, stream=True)
    for r in res:
        boxes = r.boxes
        for box in boxes:
            coordinates = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            bounding_box(img, coordinates, class_id)

    #img = cv2.resize(img, (1280, 720))
    output_media.write(img)
    #cv2.imshow("Image", img)
    #cv2.waitKey(1)
media.release()
output_media.release()
cv2.destroyWindow()


0: 384x640 (no detections), 25.4ms
Speed: 2.3ms preprocess, 25.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.2ms
Speed: 2.6ms preprocess, 25.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.1ms
Speed: 2.5ms preprocess, 25.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.1ms
Speed: 2.2ms preprocess, 25.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.7ms
Speed: 2.1ms preprocess, 20.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.6ms
Speed: 2.3ms preprocess, 20.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.3ms
Speed: 2.1ms preprocess, 21.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.6ms
Speed: 2.2ms preprocess, 20.6ms i

KeyboardInterrupt: 